# FRED ETL

In this notebook data is extracted from the Federal Reserve Bank of St. Louis and loaded into a MongoDB database.

Series that we will extract are the following:

* Producer Price Index by Commodity: Lumber and Wood Products: Lumber (WPU081)
    * https://fred.stlouisfed.org/series/WPU081
* Median Sales Price of Houses Sold for the United States (MSPUS)
    * https://fred.stlouisfed.org/series/MSPUS
* Homeownership Rate for the United States (RHORUSQ156N)
    * https://fred.stlouisfed.org/series/RHORUSQ156N


In [19]:
#import dependencies
import pandas as pd
import numpy as np
import pymongo
from fredapi import Fred
from config import fred_key
fred = Fred(api_key=fred_key)

### Extract and Transform

In [20]:
lumber_price_index = pd.DataFrame(fred.get_series('WPU081')) #index where 1982 = 100
lumber_price_index.reset_index(inplace = True)
lumber_price_index.columns = ["date","lumber_price_index"]
print(f"Number of rows: {len(lumber_price_index)}")
lumber_price_index.tail()

Number of rows: 1143


,date,lumber_price_index
1138,2020-11-01,264.2
1139,2020-12-01,289.0
1140,2021-01-01,320.9
1141,2021-02-01,341.9
1142,2021-03-01,367.1


In [21]:
median_house_price = pd.DataFrame(fred.get_series('MSPUS'))
median_house_price.reset_index(inplace = True)
median_house_price.columns = ["date","median_house_price"]
print(f"Number of rows: {len(median_house_price)}")
median_house_price.tail()

Number of rows: 233


,date,median_house_price
228,2020-01-01,329000.0
229,2020-04-01,322600.0
230,2020-07-01,337500.0
231,2020-10-01,358700.0
232,2021-01-01,347500.0


In [22]:
ownership_rate = pd.DataFrame(fred.get_series('RHORUSQ156N'))
ownership_rate.reset_index(inplace = True)
ownership_rate.columns = ["date","homeownership_rate"]
print(f"Number of rows: {len(ownership_rate)}")
ownership_rate.tail()

Number of rows: 225


,date,homeownership_rate
220,2020-01-01,65.3
221,2020-04-01,67.9
222,2020-07-01,67.4
223,2020-10-01,65.8
224,2021-01-01,65.6


In [24]:
df = pd.merge(lumber_price_index,(pd.merge(ownership_rate, median_house_price, on = "date")), on = "date", how = "right")

In [31]:
df_dictionary = df.to_dict(orient = "list")
df_dictionary

{'date': [Timestamp('1965-01-01 00:00:00'),
  Timestamp('1965-04-01 00:00:00'),
  Timestamp('1965-07-01 00:00:00'),
  Timestamp('1965-10-01 00:00:00'),
  Timestamp('1966-01-01 00:00:00'),
  Timestamp('1966-04-01 00:00:00'),
  Timestamp('1966-07-01 00:00:00'),
  Timestamp('1966-10-01 00:00:00'),
  Timestamp('1967-01-01 00:00:00'),
  Timestamp('1967-04-01 00:00:00'),
  Timestamp('1967-07-01 00:00:00'),
  Timestamp('1967-10-01 00:00:00'),
  Timestamp('1968-01-01 00:00:00'),
  Timestamp('1968-04-01 00:00:00'),
  Timestamp('1968-07-01 00:00:00'),
  Timestamp('1968-10-01 00:00:00'),
  Timestamp('1969-01-01 00:00:00'),
  Timestamp('1969-04-01 00:00:00'),
  Timestamp('1969-07-01 00:00:00'),
  Timestamp('1969-10-01 00:00:00'),
  Timestamp('1970-01-01 00:00:00'),
  Timestamp('1970-04-01 00:00:00'),
  Timestamp('1970-07-01 00:00:00'),
  Timestamp('1970-10-01 00:00:00'),
  Timestamp('1971-01-01 00:00:00'),
  Timestamp('1971-04-01 00:00:00'),
  Timestamp('1971-07-01 00:00:00'),
  Timestamp('1971-10

### Load

In [32]:
#establish MongoDB connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.housing_db

#load in lumber price index
db.fred.drop()
db.fred.find()

db.fred.insert_one(df_dictionary)